# Assay Binding analysis
Analyze how well my PCR primers and probes match the sequences I've found in my samples.

## Initialization, configuration and utility functions

In [1]:
from Bio.Seq import Seq
from Bio import SeqIO
from Bio import Align

import RCUtils

# For now this is just Enterovirus primers, but could expand it to other species
primers = RCUtils.readPrimers("qPCRPrimers.fasta")
print("Read %i primers" % (len(primers)))    

Read 17 primers


# S28 - Rhinovirus A-23

In [43]:
def printSeqBinding(path):
    global a
    # TODO: Try to print a semi-global alignment. Can use global with end_gap_score=0
    aligner = Align.PairwiseAligner(mode='local', match_score=1, mismatch_score=0, gap_score=-1)
    record = SeqIO.read(path, "fastq")
    hits = RCUtils.computePrimerHits(record, primers, allowOverlaps=True)
    for hit in hits:
        print ("%s len=%d match=%d%%" % (hit.primer.id, len(hit.primer.seq), 100*hit.mr))
        if hit.mr < 1:
            a = aligner.align(record.seq, hit.primer.seq, strand="-" if hit.rev else "+")[0]
            if a.coordinates[0][0] == 0 and a.coordinates[1][0] > 0:
                print ("  Primer %d off the start of the sequence" % a.coordinates[1][0])
            pt = len(hit.primer.seq) - a.coordinates[1][-1]
            if a.coordinates[0][-1] == len(record.seq) and pt > 0:
                print ("  Primer %d off the end of the sequence" % pt)
            print(a)
        
printSeqBinding("myseqs/S28-RVA-23.fastq")

ENTng-f.1 len=19 match=100%
ENTng-p.6 len=18 match=100%
ENTrc-f1 len=21 match=100%
ENTrc-r len=21 match=100%
HRVma-f len=19 match=100%
HRVma-p len=18 match=100%
ENTng-r len=22 match=95%
  Primer 1 off the end of the sequence
target          371 CTACTTTGGGTGTCCGTGTTT 392
                  0 |||||||||||||||||||||  21
query             0 CTACTTTGGGTGTCCGTGTTT  21

HRVma-r len=20 match=95%
target          359 GGATGGGACCAACTACTTTG 379
                  0 ||||||||||.|||||||||  20
query             0 GGATGGGACCGACTACTTTG  20

